In [ ]:
# Install packages
!pip install geopandas
!pip install datatable
!pip install gee_subset
!pip install geextract

In [27]:
# Load packages
import ee
import pandas as pd
from datetime import datetime, date, time, timedelta
from dateutil.relativedelta import *
import geopandas as gpd
import numpy as np
import datatable as dt
from datatable import f
import time
from pprint import pprint
import os, re
from gee_subset import gee_subset
import shapely.geometry
import re
import geemap
from geextract import ts_extract

In [2]:
# Trigger the authentication flow.
#ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [3]:
# Import the plot geosjon with the centroids
plots = gpd.read_file("data/plot_data/Ghana_plot_centroid.GeoJSON")
plots_dt = dt.fread("data/plot_data/Ghana_plot_centroid.csv")

In [4]:
# Create columns with x and y coordinates
plots['lon'] = plots['geometry'].x
plots['lat'] = plots['geometry'].y

In [5]:
x_dt = dt.Frame(plots['lon'])
y_dt = dt.Frame(plots['lat'])
plots_dt.cbind(x_dt, y_dt)

In [6]:
# Import the plot geosjon with the polygon geometries
plotsPol = gpd.read_file("data/plot_data/Ghana_plots_all.GeoJSON")
plotsPol_dt = dt.fread("data/plot_data/Ghana_plots_all.csv")

In [ ]:
# Turn plotsPol into GEE feature collection
g = [i for i in plotsPol.geometry]
features=[]
for i in range(len(g)):
  g = [i for i in plotsPol.geometry]
  x,y = g[i].exterior.coords.xy
  cords = np.dstack((x,y)).tolist()

  g=ee.Geometry.Polygon(cords)
  feature = ee.Feature(g)
  features.append(feature)

ee_object = ee.FeatureCollection(features)  

In [7]:
# Create ee.Multipoint (for GPM extraction)
geom_list = []
for i in range(x_dt.nrows):
  point = [x_dt[i, 'lon'], y_dt[i, 'lat']]
  geom_list.append(point)

centroid_multi = ee.Geometry.MultiPoint(geom_list)

In [7]:
# Time series settings
start_date = datetime(2013, 1, 1)
end_date = datetime.today()

**Extract sentinel-2 (using gee_subset)**

In [ ]:
refl_s2 = dt.Frame()

for pl in range(plots_dt.nrows):
  print(pl)

  s2_out = gee_subset.gee_subset(product = "COPERNICUS/S2_SR", 
                                 bands = ["B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B9", "B11", "B12", "QA60"], 
                                 start_date = start_ts, end_date = end_ts, 
                                 latitude = plots_dt[pl, 'lat'], longitude = plots_dt[pl,'lon'], 
                                 scale = 10)
  # Transform to dt
  s2_out = dt.Frame(s2_out)
  
  # Create column with plotID 
  s2_out['plotID'] = plots_dt[pl, 'plotID']

  refl_s2.rbind(s2_out)

In [ ]:
# Write reflectance time series to drive
refl_s2.to_csv("/content/drive/MyDrive/Colab Notebooks/Regreening data/refl_s2.csv")

In [ ]:
# Read the Sentinel 2 reflectance data
refl_s2 = dt.fread("/content/drive/MyDrive/Colab Notebooks/Regreening data/refl_s2.csv")

**Extract L8 polygons**

In [54]:
# Select bands and resolution to extract from landsat 8
#bands = ["SR_B1", "SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B6", "SR_B7", "QA_PIXEL", "ST_CDIST", "QA_RADSAT"]
bands = ["SR_B4", "SR_B5", "SR_B6", "QA_PIXEL"]
scale = 30
product='LANDSAT/LC08/C02/T1_L2'

In [55]:
# Create function to get yearly dates for the yearly extraction
def get_year_dates(start, end):
    yearly_dates = [start]
    year_diff = (end.year - start.year) + 1

    for i in range(1, year_diff+1):
        next_year = start_date + relativedelta(years =+ i) 
        yearly_dates.append(next_year)

    return yearly_dates

year_dates = get_year_dates(start_date, end_date)

In [56]:
#refl_l8_pol = dt.Frame()
for pl in range(21781, len(plotsPol)):
    print(pl)

    g = [i for i in plotsPol.geometry]
    x,y = g[pl].exterior.coords.xy
    cords = np.dstack((x,y)).tolist()
    geometry = ee.Geometry.Polygon(cords)
    geometry = geometry.buffer(0.000063)

    # Check if geometry exceeds critical size (15ha)
    if plotsPol['Hectare'].iloc[pl] > 40:
        print('Area too large --> extracting per year')
        # Extract data per year to prevent GEE size error
        for yr in range(len(year_dates)-1):
            try:
                col = ee.ImageCollection(product).\
                    select(tuple(bands)).\
                    filterDate(year_dates[yr], year_dates[yr+1]).filterBounds(geometry)


                # Make a df
                region = col.getRegion(geometry, int(scale)).getInfo()
                df = pd.DataFrame.from_records(region[1:len(region)])
                df.columns = region[0]
  
                df.time = df.time / 1000
                df['time'] = pd.to_datetime(df['time'], unit = 's')
                df.rename(columns = {'time': 'date'}, inplace = True)
                df.sort_values(by = 'date')

                df['product'] = pd.Series(product, index = df.index)

                # Transform to dt
                l8_out = dt.Frame(df)

                # Create column with plotID 
                l8_out['plotID'] = plotsPol['plotID'].iloc[pl]
                refl_l8_pol.rbind(l8_out)

            except:
                pass
                
    else:
        col = ee.ImageCollection(product).\
            select(tuple(bands)).\
            filterDate(start_date, end_date).filterBounds(geometry)

        region = col.getRegion(geometry, int(scale)).getInfo()

        # If no pixels in geometry, take centroid of plot
        if len(region) == 1:
            print('Not enough pixels in geometry (taking centroid)')
            
            geometry = ee.Geometry.Point([plots_dt[pl,'lon'], plots_dt[pl, 'lat']])
            col = ee.ImageCollection(product).\
                select(tuple(bands)).\
                filterDate(start_date, end_date).filterBounds(geometry)
      
        region = col.getRegion(geometry, int(scale)).getInfo()
        df = pd.DataFrame.from_records(region[1:len(region)])
        df.columns = region[0]
  
        df.time = df.time / 1000
        df['time'] = pd.to_datetime(df['time'], unit = 's')
        df.rename(columns = {'time': 'date'}, inplace = True)
        df.sort_values(by = 'date')

        df['product'] = pd.Series(product, index = df.index)

        # Transform to dt
        l8_out = dt.Frame(df)

        # Create column with plotID 
        l8_out['plotID'] = plotsPol['plotID'].iloc[pl]

        refl_l8_pol.rbind(l8_out)

21781
Area too large --> extracting per year
21782
Area too large --> extracting per year
21783
Area too large --> extracting per year
21784
Area too large --> extracting per year
21785
Area too large --> extracting per year
21786
Area too large --> extracting per year
21787
Area too large --> extracting per year
21788
Area too large --> extracting per year
21789
Area too large --> extracting per year
21790
Area too large --> extracting per year
21791
Area too large --> extracting per year
21792
Area too large --> extracting per year
21793
Area too large --> extracting per year
21794
Area too large --> extracting per year
21795
Area too large --> extracting per year
21796
Area too large --> extracting per year
21797
Area too large --> extracting per year
21798
Area too large --> extracting per year
21799
Area too large --> extracting per year
21800
Area too large --> extracting per year
21801
Area too large --> extracting per year
21802
Area too large --> extracting per year
21803
Area

In [58]:
refl_l8_pol.to_csv("output/gee/Ghana_refl_l8_pol_21655-end.csv")

In [53]:
refl_l8_pol = dt.fread("output/gee/Ghana_refl_l8_pol_21655-21780.csv")

**Extract S1 polygons**

In [8]:
# Settings and parameters
product = "COPERNICUS/S1_GRD"
bands = 'VV'
start_date = start_ts
end_date = end_ts
instrument = 'IW'
orbit = 'ASCENDING'
scale = 10
sar_band = ''.join(bands)
g = [i for i in plotsPol.geometry]

# Time series settings
start_date=datetime(2013, 1, 1)
end_date=datetime.today()

In [9]:
# Create function to get yearly dates for the yearly extraction
def get_year_dates(start, end):
    yearly_dates = [start]
    year_diff = (end.year - start.year) + 1

    for i in range(1, year_diff+1):
        next_year = start_date + relativedelta(years =+ i) 
        yearly_dates.append(next_year)

    return yearly_dates

year_dates = get_year_dates(start_date, end_date)

In [31]:
#refl_s1_pol = dt.Frame()
for pl in range(21642, len(plotsPol)):
    print(pl)

    g = [i for i in plotsPol.geometry]
    x,y = g[pl].exterior.coords.xy
    cords = np.dstack((x,y)).tolist()
    geometry = ee.Geometry.Polygon(cords)
    geometry = geometry.buffer(0.000063)

    # Check if geometry exceeds critical size (15ha)
    if plotsPol['Hectare'].iloc[pl] > 15:
        print('Area too large --> extracting per year')
        # Extract data per year to prevent GEE size error
        for yr in range(len(year_dates)-1):
            try:
                col = ee.ImageCollection('COPERNICUS/S1_GRD').\
                    filter(ee.Filter.listContains('transmitterReceiverPolarisation', sar_band)).\
                    filter(ee.Filter.eq('instrumentMode', instrument)).select(bands).\
                    filter(ee.Filter.eq('resolution_meters', int(scale))).\
                    filter(ee.Filter.eq('orbitProperties_pass', orbit)).\
                    filterDate(year_dates[yr], year_dates[yr+1]).filterBounds(geometry)
                # Make a df
                region = col.getRegion(geometry, int(scale)).getInfo()
                df = pd.DataFrame.from_records(region[1:len(region)])
                df.columns = region[0]
  
                df.time = df.time / 1000
                df['time'] = pd.to_datetime(df['time'], unit = 's')
                df.rename(columns = {'time': 'date'}, inplace = True)
                df.sort_values(by = 'date')

                df['product'] = pd.Series(product, index = df.index)

                # Transform to dt
                s1_out = dt.Frame(df)

                # Create column with plotID 
                s1_out['plotID'] = plotsPol['plotID'].iloc[pl]
                refl_s1_pol.rbind(s1_out)

            except:
                pass
                
    else:
        col = ee.ImageCollection('COPERNICUS/S1_GRD').\
            filter(ee.Filter.listContains('transmitterReceiverPolarisation', sar_band)).\
            filter(ee.Filter.eq('instrumentMode', instrument)).select(bands).\
            filter(ee.Filter.eq('resolution_meters', int(scale))).\
            filter(ee.Filter.eq('orbitProperties_pass', orbit)).\
            filterDate(start_date, end_date).filterBounds(geometry)

        region = col.getRegion(geometry, int(scale)).getInfo()

        # If no pixels in geometry, take centroid of plot
        if len(region) == 1:
            print('Not enough pixels in geometry (taking centroid)')
            geometry = ee.Geometry.Point([plots_dt[pl,'lon'], plots_dt[pl, 'lat']])
            col = ee.ImageCollection('COPERNICUS/S1_GRD').\
                filter(ee.Filter.listContains('transmitterReceiverPolarisation', sar_band)).\
                filter(ee.Filter.eq('instrumentMode', instrument)).select(bands).\
                filter(ee.Filter.eq('resolution_meters', int(scale))).\
                filter(ee.Filter.eq('orbitProperties_pass', orbit)).\
                filterDate(start_date, end_date).filterBounds(geometry)
      
        region = col.getRegion(geometry, int(scale)).getInfo()
        df = pd.DataFrame.from_records(region[1:len(region)])
        df.columns = region[0]
  
        df.time = df.time / 1000
        df['time'] = pd.to_datetime(df['time'], unit = 's')
        df.rename(columns = {'time': 'date'}, inplace = True)
        df.sort_values(by = 'date')

        df['product'] = pd.Series(product, index = df.index)

        # Transform to dt
        s1_out = dt.Frame(df)

        # Create column with plotID 
        s1_out['plotID'] = plotsPol['plotID'].iloc[pl]

        refl_s1_pol.rbind(s1_out)

21642
Area too large --> extracting per year
21643
Area too large --> extracting per year
21644
Area too large --> extracting per year
21645
Area too large --> extracting per year
21646
Area too large --> extracting per year
21647
Area too large --> extracting per year
21648
Area too large --> extracting per year
21649
Area too large --> extracting per year
21650
Area too large --> extracting per year
21651
Area too large --> extracting per year
21652
Area too large --> extracting per year
21653
Area too large --> extracting per year
21654
Area too large --> extracting per year
21655
Area too large --> extracting per year
21656
Area too large --> extracting per year
21657
Area too large --> extracting per year
21658
Area too large --> extracting per year
21659
Area too large --> extracting per year
21660
Area too large --> extracting per year
21661
Area too large --> extracting per year
21662
Area too large --> extracting per year
21663
Area too large --> extracting per year
21664
Area

In [ ]:
# Take average backscatter per plot
refl_s1_pol[:, dt.mean(dt.f.VV), dt.by("plotID", "date")]

In [32]:
refl_s1_pol.to_csv("output/gee/Ghana_refl_s1_pol.csv")

In [ ]:
refl_s1_pol = dt.fread("output/gee/Ghana_refl_s1_pol_0-21641.csv")

**Extract Precipitation data (GPM)**

In [10]:
## Set parameters for the time series
start_ts = datetime(2013, 1, 1)
end_ts = datetime.today()

# Specify number of days in period of interest
d0 = datetime(start_ts.year, start_ts.month, start_ts.day)
d1 = datetime(end_ts.year, end_ts.month, 1)
delta = d1 - d0
days = delta.days

# number of months in period
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month
months_ts = diff_month(d1, d0)

# Create list with the dates off all the observations
months_date = []
for m in range(months_ts):
  first_month = start_ts
  next_month = first_month + relativedelta(months =+ m)
  months_date.append(next_month)

In [11]:
# Extract all the precipitation data for all sites
gpm = ee.ImageCollection('NASA/GPM_L3/IMERG_V06').\
       select('precipitationCal').\
       filterDate(start_ts, end_ts).filterBounds(centroid_multi)

In [12]:
# Create a function to go over the FeatureCollection and take the monthly sum
def GPMsum(img_collection):
  mylist = ee.List([])
  for m in range(months_ts):

    ini = start_ts + relativedelta(months=+m)
    end = ini + relativedelta(months=+1) + relativedelta(days=-1)

    sum_image = img_collection.filterDate(ini,end).select(0).sum()
    mylist = mylist.add(sum_image.set('system:time_start', ini))
  return ee.ImageCollection.fromImages(mylist)

In [13]:
# Apply the 'GPMsum' function to create FeatureCollection with monthly sum 
monthlyGPM = ee.ImageCollection(GPMsum(gpm))
# Sort FeatureCollection by date and create single image with bands
monthlyGPM_stack = monthlyGPM.sort('system:time_start').toBands().multiply(0.5)

In [ ]:
# Extract the monthly GPM value for each site and put in a df (takes about 30/60 min)
#monthlyGPM_df = pd.DataFrame(columns=['Name', 'GPM/month', 'plotID']) 
#stopped at 7224
for pl in range(7225, plots_dt.nrows):
  
  geometry = ee.Geometry.Point([plots_dt[pl,'lon'], plots_dt[pl, 'lat']])

  # Access GPM value at site location
  monthlyGPM_site = monthlyGPM_stack.reduceRegion(ee.Reducer.first(), geometry, 11000, bestEffort=True)
  monthlyGPM_site = monthlyGPM_site.getInfo() # Extract value

  # Store in df and assign plotID
  gpm_site_df = pd.DataFrame(monthlyGPM_site.items(), columns=['Name', 'GPM/month'])
  gpm_site_df['plotID'] = plots_dt[pl, 'plotID']

  monthlyGPM_df = monthlyGPM_df.append(gpm_site_df)

  print(pl)

In [14]:
img_todrive = {
    #'description': 'gpm_rwanda_'+str(startyear)+"_"+str(endyear),
    'folder': 'Regreening_Africa',
    'scale': 11000,
    'maxPixels': 1e13,
    'region': centroid_multi,
    'fileFormat': 'GeoTIFF'}

task = ee.batch.Export.image.toDrive(monthlyGPM_stack, 'Rwanda_GPM_stack', **img_todrive)
task.start()

In [ ]:
## Assign dates to all the GPM measurements
# Sort the date in months_date according to monthlyGPM_df
months_sort = []
for i in range(len(months_date)):
  string = monthlyGPM_df['Name'].iloc[i]
  month_nr = int(re.findall(r'\d+', string)[0])
  months_sort.append(months_date[month_nr])

# Repeat list n times and add to df
months_sort = months_sort*plots_dt.nrows
months_df = pd.DataFrame(months_sort, columns = ['Date'])
monthlyGPM_df = monthlyGPM_df.join(months_df)

In [ ]:
# Write GPM time series to drive
monthlyGPM_df.to_csv("/content/drive/MyDrive/Colab Notebooks/Regreening data/GPM_monthly.csv")

In [ ]:
# Read the Landsat 9 reflectance data
#monthlyGPM_df = dt.fread("/content/drive/MyDrive/Colab Notebooks/Regreening data/GPM_monthly.csv")
monthlyGPM_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Regreening data/GPM_monthly.csv")